In [1]:
from lyra.parser import parse_form
from lyra.transformer.transformer import transform_form
from IPython.display import display, Markdown
from lyra.translator.translate import make_collection, imports, make_mapping, make___all__

from fundar import json
from glob import glob
from tqdm.auto import tqdm

forms = glob('../data/*-form.json')

ALL = None

def normalize_name(x):
    return x.replace('-', '_')

def display_markdown(md):
    display(Markdown(md))

names = [None for _ in forms]
#with open('lyra/cds_collections/all_collections.py', 'w') as f:
for i, form in enumerate(forms):
    collection_name = form.split('-form')[0].rsplit('/')[-1]
    normalized_name = normalize_name(collection_name)
    with open(f'lyra/cds_collections/collection_{normalized_name}.py', 'w') as f:
        f.write(imports)
        ast = parse_form(json.load(form))
        names[i] = collection_name

        transformed_ast = sorted(set(transform_form(ast)),
                                    key=lambda x: (x.has_default_value(), x.name))
        
        #display_str.append('\n```')

        f.write(make_collection(collection_name, transformed_ast, normalizer=normalize_name))

        # f.write('\n\n')
        # f.write(make_mapping(sorted(names), normalizer=normalize_name))
        # f.write('\n\n')
        # ALL = make___all__(sorted(names), normalizer=normalize_name, extra=['cds_collections'])
        # f.write(ALL)

# PREFIX = 'download_'
PREFIX = 'Collection_'
with open('lyra/cds_collections/__init__.py', 'w') as f:
    f.write("from lyra.translator.types import Collection\n")
    f.write("from dataclasses import dataclass\n")
    f.write("from typing import Dict, Callable\n\n")

    for x in names:
        f.write(f'from .collection_{normalize_name(x)} import {PREFIX}{normalize_name(x)}\n')

    dictitems = '\n'.join([f'{normalize_name(x)} = {PREFIX}{normalize_name(x)}' for x in names])
    f.write(
f"""
{dictitems}
"""
    )
#     for x in ALL[11:-1].replace('"', '').replace("'", '').split(', \n'):
#         f.write(f'from .all_collections import {x}\n')